In [17]:
// Run this boilerplate for the necessary imports

import $ivy.`edu.berkeley.cs::chisel3:3.0-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::firrtl:1.0-SNAPSHOT_2017-07-19`
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import scala.collection.mutable
def getVerilog[T <: Module](gen: => T): String = {
  chisel3.Driver.execute(Array[String](), {() => gen}) match {
    case ChiselExecutionSuccess(_, _, Some(firrtl.FirrtlExecutionSuccess(_, verilog))) => verilog
  }
}

import $ivy.$                                                 

import $ivy.$                                                          

import $ivy.$                                                

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import scala.collection.mutable

defined function getVerilog

In [20]:
class MyManyElementFir(consts: Seq[Int]) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  // Reference solution
  val regs = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) regs += io.in
      else       regs += RegNext(regs(i - 1), 0.U)
  }
  
  val muls = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      muls += regs(i) * consts(i).U(8.W)
  }

  val scan = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) scan += muls(i)
      else scan += muls(i) + scan(i - 1)
  }

  io.out := scan(consts.length - 1)
}

defined class MyManyElementFir

In [36]:
// Simple 4-point moving average
Driver(() => new MyManyElementFir(Seq(1, 1, 1, 1))) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1, 0, 0, 0
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 5)  // 4, 1, 0, 0
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 8)  // 3, 4, 1, 0
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 10)  // 2, 3, 4, 1
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 16)  // 7, 2, 3, 4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 12)  // 0, 7, 2, 3
  }
}
println(chisel3.Driver.emit(() => new MyManyElementFir(Seq(1, 1, 1, 1))))

[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 9.3 ms
Total FIRRTL Compile Time: 8.0 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1507918639968
test cmd19WrapperHelperMyManyElementFir Success: 6 tests passed in 10 cycles taking 0.003722 seconds
[info] [0.002] RAN 5 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
;buildInfoPackage: chisel3, version: 3.0-SNAPSHOT_2017-07-19, scalaVersion: 2.11.11, sbtVersion: 0.13.15, builtAtString: 2017-07-19 18:56:34.453, builtAtMillis: 1500490594453
circuit cmd19WrapperHelperMyManyElementFir : 
  module cmd19WrapperHelperMyManyElementFir : 
    input clock : Clock
    input reset : UInt<1>
    output io : {flip in : UInt<8>, out : UInt<8>}
    
    clock is invalid
    reset is invalid
    io is invalid
    reg regs_1 : UInt, clock with : (reset => (reset, UInt<1>("h00"))) @[cmd19.sc 11:33]
    regs_1 <= io.in @[cmd19.sc 11:33]
    reg regs_2 

res35_0: Boolean = true

In [23]:
class MyManyDynamicElementFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })

  // Reference solution
  val regs = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until length) {
      if(i == 0) regs += io.in
      else       regs += RegNext(regs(i - 1), 0.U)
  }
  
  val muls = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until length) {
      muls += regs(i) * io.consts(i)
  }

  val scan = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until length) {
      if(i == 0) scan += muls(i)
      else scan += muls(i) + scan(i - 1)
  }

  io.out := scan(length - 1)
}

defined class MyManyDynamicElementFir

In [25]:
// Simple 4-point moving average
Driver(() => new MyManyDynamicElementFir(4)) {
  c => new PeekPokeTester(c) {
    /* Set constant values */
    poke(c.io.consts(0), 1)
    poke(c.io.consts(1), 1)
    poke(c.io.consts(2), 1)
    poke(c.io.consts(3), 1)
    
    /* Set input value and check output value */
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1, 0, 0, 0
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 5)  // 4, 1, 0, 0
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 8)  // 3, 4, 1, 0
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 10)  // 2, 3, 4, 1
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 16)  // 7, 2, 3, 4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 12)  // 0, 7, 2, 3
  }
}

[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 10.4 ms
Total FIRRTL Compile Time: 8.7 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1507917164324
test cmd22WrapperHelperMyManyDynamicElementFir Success: 6 tests passed in 10 cycles taking 0.005294 seconds
[info] [0.004] RAN 5 CYCLES PASSED


res24: Boolean = true

In [32]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })

  // Reference solution
  val regs = RegInit(Vec(length - 1, UInt(8.W)), Vec.fill(length - 1)(0.U))
  for(i <- 0 until length - 1) {
      if(i == 0) regs(i) := io.in
      else       regs(i) := regs(i - 1)
  }
  
  val muls = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) muls(i) := io.in * io.consts(i)
      else       muls(i) := regs(i - 1) * io.consts(i)
  }

  val scan = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) scan(i) := muls(i)
      else scan(i) := muls(i) + scan(i - 1)
  }

  io.out := scan(length - 1)
}

defined class MyManyDynamicElementVecFir

In [35]:
// Simple 4-point moving average
Driver(() => new MyManyDynamicElementVecFir(4)) {
  c => new PeekPokeTester(c) {
    /* Set constant values */
    poke(c.io.consts(0), 1)
    poke(c.io.consts(1), 1)
    poke(c.io.consts(2), 1)
    poke(c.io.consts(3), 1)
    
    /* Set input value and check output value */
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1, 0, 0, 0
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 5)  // 4, 1, 0, 0
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 8)  // 3, 4, 1, 0
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 10)  // 2, 3, 4, 1
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 16)  // 7, 2, 3, 4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 12)  // 0, 7, 2, 3
  }
}
println(chisel3.Driver.emit(() => new MyManyDynamicElementVecFir(4)))

[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 10.7 ms
Total FIRRTL Compile Time: 8.3 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1507918487575
test cmd31WrapperHelperMyManyDynamicElementVecFir Success: 6 tests passed in 10 cycles taking 0.004068 seconds
[info] [0.003] RAN 5 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
;buildInfoPackage: chisel3, version: 3.0-SNAPSHOT_2017-07-19, scalaVersion: 2.11.11, sbtVersion: 0.13.15, builtAtString: 2017-07-19 18:56:34.453, builtAtMillis: 1500490594453
circuit cmd31WrapperHelperMyManyDynamicElementVecFir : 
  module cmd31WrapperHelperMyManyDynamicElementVecFir : 
    input clock : Clock
    input reset : UInt<1>
    output io : {flip in : UInt<8>, out : UInt<8>, flip consts : UInt<8>[4]}
    
    clock is invalid
    reset is invalid
    io is invalid
    wire _T_21 : UInt<1>[3] @[cmd31.sc 9:70]
    _T_21 is invalid @[cmd31.sc 9:70]
  

res34_0: Boolean = true